In [1]:
import gender_guesser.detector as gg
import matplotlib as plt
import pandas as pd

In [2]:
data = pd.read_csv('data/imdb-directors-writers-cast.csv', index_col=0)
data.head()

,year,title,imdbid,id,rating,directors,writers,cast
0,1984,C.H.U.D.,87015.0,9327,2,Douglas Cheek,Parnell Hall\nShepard Abbott\nChristopher Curr...,John Heard\nDaniel Stern\nChristopher Curry\nK...
1,1955,Les grandes manoeuvres,48133.0,7878,3,René Clair,René Clair\nJérôme Géronimi\nJean Marsan,Michèle Morgan\nGérard Philipe\nJean Desailly\...
2,1964,King &amp; Country,58263.0,3250,0,Joseph Losey,Evan Jones\nJohn Wilson\nJ.L. Hodson\nDirk Bog...,Dirk Bogarde\nTom Courtenay\nLeo McKern\nBarry...
3,2002,"Coast Guard, The",341384.0,3163,1,Kim Ki-duk,Kim Ki-duk,Jang Dong-Gun\nJeong-hak Kim\nJi-a Park\nHae-J...
4,1965,Daikaiju Gamera,59080.0,6428,2,Noriaki Yuasa,Niisan Takahashi,Eiji Funakoshi\nHarumi Kiritachi\nJunichiro Ya...


In [3]:
len(data['directors'].dropna())

8464

In [4]:
data.dropna(inplace=True)

In [5]:
data[data['directors'].apply(lambda x: len(x.split('\n'))) > 10].head()

,year,title,imdbid,id,rating,directors,writers,cast
102,2010,Halo: Legends,1480660.0,7927,1,Frank O'Connor\nShinji Aramaki\nHideki Futamur...,Joseph Staten\nFrank O'Connor\nHiroshi Yamazak...,Josh Grelle\nBrittney Karbowski\nLuci Christia...
358,2020,Motherland: Fort Salem,9900092.0,10010,3,Amanda Tapping\nSteven A. Adelson\nShannon Koh...,Eliot Laurence\nEliot Laurence\nEliot Laurence...,Taylor Hickson\nAmalia Holm\nDemetria McKinney...
852,2013,Orphan Black,2234222.0,5037,3,John Fawcett\nDavid Frazee\nKen Girotti\nT.J. ...,Kim Coghill\nAndrew De Angelis\nJeff Detsky\nJ...,Tatiana Maslany\nJordan Gavaris\nMaria Doyle K...
983,2011,Scott and Bailey,1843678.0,6847,3,Morag Fullarton\nChina Moo-Young\nPaul Walker\...,Sally Wainwright\nSally Wainwright\nSally Wain...,Suranne Jones\nLesley Sharp\nTony Mooney\nDavi...
1256,2018,Timeless,5511582.0,10165,3,Greg Beeman\nJohn F. Showalter\nHolly Dale\nCh...,Eric Kripke\nEric Kripke\nShawn Ryan\nShawn Ry...,Abigail Spencer\nMatt Lanter\nMalcolm Barrett\...


In [6]:
gender_to_score = {'male': 1,
                   'mostly_male': 0.5,
                   'andy': 0,
                   'unknown': 0,
                   'mostly_female': -0.5,
                   'female': -1}

In [7]:
detector = gg.Detector()

In [8]:
def gender_score(string):
    list_first_names = string.split('\n')
    list_first_names = [name.split(' ')[0] for name in list_first_names]
    list_genders = [detector.get_gender(firstname) for firstname in list_first_names]
    scores = [gender_to_score[gender] for gender in list_genders]
    return scores

In [9]:
df = data.copy()

for col in ('directors', 'writers', 'cast'):
    df[col] = data[col].apply(gender_score)
    df[col + '_male'] = df[col].apply(lambda x: sum([el for el in x if el > 0]))
    df[col + '_female'] = df[col].apply(lambda x: sum([-el for el in x if el < 0]))
    df[col] = df[col].apply(sum)

In [10]:
df

,year,title,imdbid,id,rating,directors,writers,cast,directors_male,directors_female,writers_male,writers_female,cast_male,cast_female
0,1984,C.H.U.D.,87015.0,9327,2,1.0,2.0,19.5,1.0,0.0,2.0,0.0,27.5,8.0
1,1955,Les grandes manoeuvres,48133.0,7878,3,1.0,3.0,14.0,1.0,0.0,3.0,0.0,32.0,18.0
2,1964,King &amp; Country,58263.0,3250,0,1.0,3.0,17.5,1.0,0.0,3.0,0.0,18.5,1.0
3,2002,"Coast Guard, The",341384.0,3163,1,-0.5,-0.5,-1.5,0.0,0.5,0.0,0.5,0.0,1.5
4,1965,Daikaiju Gamera,59080.0,6428,2,1.0,0.0,29.0,1.0,0.0,0.0,0.0,33.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8470,1954,Gojira,47034.0,5505,1,0.0,4.0,12.0,0.0,0.0,4.0,0.0,17.0,5.0
8471,2005,"Devil&#39;s Rejects, The",395584.0,3628,3,1.0,2.0,20.0,1.0,0.0,2.0,0.0,30.0,10.0
8472,2008,Solo Quiero Caminar,1078600.0,7381,3,1.0,1.0,21.5,1.0,0.0,1.0,0.0,27.5,6.0
8473,2009,Le h&eacute;risson,1442519.0,2439,3,-1.0,-2.0,-8.0,0.0,1.0,0.0,2.0,5.0,13.0


In [11]:
df.to_csv('data/directors_writers_cast_score.csv', index=False)

In [ ]:
df.hist(column='directors', bins=50)

In [ ]:
df_bis = df[['rating', 'directors',  'writers', 'cast']]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
for col in ('directors', 'writers', 'cast'):
    df_bis[col] = sc.fit_transform(df_bis[[col]])

In [ ]:
df_bis

In [ ]:
df_bis.corr()